In [ ]:
from pynq import Overlay, MMIO
import time
import random
import numpy as np


bubble_sort_code = [
    0x42000437, 0x02000493, 0x00000293, 0x04928263, 
    0x00000313, 0x40548933, 0xfff90913, 0x03230663, 
    0x00231393, 0x008383b3, 0x0003ae03, 0x0043ae83, 
    0x01ceaf33, 0x000f0663, 0x01d3a023, 0x01c3a223, 
    0x00130313, 0xfd9ff06f, 0x00128293, 0xfc1ff06f, 
    0x42002fb7, 0xffcf8f93, 0xdeadcf37, 0xeeff0f13, 
    0x01efa023, 0x0000006f
]

def run_test(bitstream_path, cpu_type="Pipelined"):
    print(f"\n--- Testing {cpu_type} Core ({bitstream_path}) ---")
    ol = Overlay(bitstream_path)
    
    # Map MMIO (Adjust addresses if they differ between designs)
    inst_mmio = MMIO(0x40000000, 0x2000)
    data_mmio = MMIO(0x42000000, 0x2000)
    gpio_mmio = MMIO(0x41200000, 0x10)

    # Prepare random data
    test_data = [random.randint(-255, 255) for _ in range(32)]
    print(f"Array for sort: {test_data}")
    
    # 1. Initialization: Hold Reset [cite: 98]
    gpio_mmio.write(0x8, 0x0) 
    
    # 2. Inject Data and Instructions [cite: 102, 107]
    for i, val in enumerate(test_data):
        data_mmio.write(i * 4, val)
    for i, code in enumerate(bubble_sort_code):
        inst_mmio.write(i * 4, code)
        
    data_mmio.write(0x1FFC, 0x0) # Reset Status Flag [cite: 108]

    # 3. Execution & Timing 
    print("Releasing Reset...")
    gpio_mmio.write(0x8, 0x1)
    
    # Use perf_counter for higher precision wall-clock timing
    start_wall = time.perf_counter()
    
    timeout = 5.0
    found_done = False
    while (time.perf_counter() - start_wall) < timeout:
        # Polling status flag [cite: 124]
        if data_mmio.read(0x1FFC) == 0xDEADBEEF:
            end_wall = time.perf_counter()
            found_done = True
            break
            
    if not found_done:
        print(f"Error: {cpu_type} timed out!")
        return None

    duration = end_wall - start_wall
    print(f"Wall-clock Time: {duration:.6f} seconds")
    
    # Verify Sorting [cite: 130]
    results = [data_mmio.read(i * 4) for i in range(32)]
    if results == sorted(test_data):
        print("Result: PASS")
        print(f"Output array: {results}")
        return duration
    else:
        print("Result: FAIL")
        print(f"Output array: {results}")
        return None

# Execute both and compare
time_pipelined = run_test("design.bit", "Pipelined")
time_single = run_test("design_single.bit", "Single-Cycle")

if time_pipelined and time_single:
    speedup = time_single / time_pipelined
    print(f"\nPerformance Gain: {speedup:.2f}x")

array: [73, 194, 52, 123, 250, 208, 133, 17, 65, 79, 93, 75, 196, 147, 226, 223, 13, 191, 172, 51, 201, 201, 114, 123, 201, 70, 128, 44, 254, 1, 68, 59]
All set !
Starting the Riscv core...
Releasing Reset signal via GPIO2...
Done signal detected! (Mem: 0xdeadbeef, HW: 1)
Time used: 0.0122s
Sorted Array: [1, 13, 17, 44, 51, 52, 59, 65, 68, 70, 73, 75, 79, 93, 114, 123, 123, 128, 133, 147, 172, 191, 194, 196, 201, 201, 201, 208, 223, 226, 250, 254]
Pass!
